In [ ]:
!pip3 install opencv-python
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip3 install matplotlib
!pip3 install numpy
!pip3 install pandas
!pip3 install pillow

In [ ]:
import matplotlib.pyplot as plt # for plotting
import numpy as np # for transformation
import csv
import os
import pandas as pd
from PIL import Image
import torch # PyTorch package
import torchvision # load datasets
import torchvision.transforms as transforms # transform data
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn # basic building block for neural neteorks
import torch.nn.functional as F # import convolution functions like Relu
import torch.optim as optim # optimzer

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.labels_frame = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.labels_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.labels_frame.iloc[idx, 0],
                                self.labels_frame.iloc[idx, 1])
        image = Image.open(img_name).convert('RGB')
        label = self.labels_frame.iloc[idx, 2]

        if self.transform:
            image = self.transform(image)

        return image, label

# Path to your CSV file
csv_file = "your datapath here"

# Define your transformations
transform = transforms.Compose([
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize the images
])
# transform = transforms.Compose([
#     transforms.ToTensor()])

# Create a custom dataset instance
custom_dataset = CustomDataset(csv_file=csv_file, transform=transform)
# Create a data loader
data_loader = torch.utils.data.DataLoader(custom_dataset, batch_size=4, shuffle=True)

classes = ('capacitor', 'led', 'none')
for images, label in data_loader:
    print(label)
    print(type(label))
    break

In [ ]:
# show imgs plz
def show_imgs(img):
    img = img / 2 + 0.5 # unormalize
    img_np = img.numpy()
    plt.imshow(np.transpose(img_np, (1,2,0)))
    plt.show()

In [ ]:
# show images for sanity
# Define a counter to keep track of the image index
image_index = 0

# Iterate over data loader
for images, labels in data_loader:
    # Show every 16th image
    for j in range(len(images)):
        if image_index % 200 == 0:
            plt.imshow(images[j].permute(1, 2, 0))  # Permute the dimensions to (height, width, channels) for displaying with imshow
            plt.show()
        image_index += 1

    # Print the labels for the entire batch

In [ ]:
class cnn(nn.Module):
    def __init__(self):
        super(cnn, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size = 3, stride = 1) 
        # self.conv11 = nn.Conv2d(32, 32, kernel_size = 3, stride = 1)
        self.pool1 = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size = 3, stride = 1)
        # self.conv22 = nn.Conv2d(64, 64, kernel_size = 3, stride = 1)
        self.pool2 = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(8198656, 64)
        self.relu1 = nn.ReLU()
        # self.fc2 = nn.Linear(120, 84)
        # self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 3)

    def forward(self, img):
        img = self.conv1(img)
        # img = self.conv11(img)
        img = self.pool1(img)
        img = self.conv2(img)
        # img = self.conv22(img)
        img = self.pool2(img)
        img = torch.flatten(img, 1)
        img = self.fc1(img)
        img = self.relu1(img)
        # img = self.fc2(img)
        # img = self.relu2(img)
        img = self.fc3(img)
        return img

cnn = cnn()

In [ ]:
loss_fxn = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr = 0.001, weight_decay = 0.005, momentum = 0.9)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
cnn.to(device)

In [ ]:
for epoch in range(51):
    model_name = str(epoch) + ".pth"
    model_save = os.path.join('your save path here', model_name)
    loss_log = 0.0
    for i, data in enumerate(data_loader,0):
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        cnn = cnn.to(device)
        optimizer.zero_grad()
        torch.cuda.empty_cache()
        outputs = cnn(images)
        loss = loss_fxn(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        loss_log += loss.item()
    if epoch in [4, 30, 40, 50]:
        torch.save(cnn.state_dict(), model_save)
    if epoch in [4, 15, 20, 25, 30, 35, 40, 45, 50]:
        print(loss_log)

print('Finished Training')


In [ ]:
# save model
path = 'your model path here'
torch.save(cnn.state_dict(), path)